In [4]:
import re

def decode_arg(input,format="r/rm32"):
    # formats: r/rmXX n/rmXX s/rmXX
    # XX=8,16,32,64
    assert re.match(r'[nrs]/rm(8|16|32|64)',format)
    r_f = format[0]
    size_f=int(format[4:])
    
    nums = []
    while len(input)>0 and input[0] not in '0123456789abcdefABCDEF':
        assert input[0]==' '
        input = input[1:]
    if type(input)==str:
        while len(input)>0:
            s = ''
            while len(input)>0 and input[0] in '0123456789abcdefABCDEF':
                s+=input[0]
                input = input[1:]
            nums.append(int(s,16))
            while len(input)>0 and input[0] not in '0123456789abcdefABCDEF':
                assert input[0]==' '
                input = input[1:]
    elif type(input)==int:
        nums.append(input)
    elif type(input)==list:
        nums = input
    else:
        raise Exception()
        
    assert len(nums)>0
    
    mod_rm=nums[0]
    mod = (mod_rm & 0b11000000)>>6
    r = (mod_rm & 0b111000)>>3
    m = mod_rm & 0b111
    print('mod r m: {:02b} {:03b} {:03b}'.format(mod,r,m))
    
    reg8 = ['al' ,'cl', 'dl', 'bl', 'ah', 'ch', 'dh', 'bh' ]
    reg16 =['ax' ,'cx', 'dx', 'bx', 'sp', 'bp', 'si', 'di' ]
    reg32 =['eax','ecx','edx','ebx','esp','ebp','esi','edi']
    reg64 =['rax','rcx','rdx','rbx','rsp','rbp','rsi','rdi']
    reg_s =['es' ,'cs' ,'ss' ,'ds' ,'fs' ,'gs' ,None ,None ]
    
    #r
    if r_f=='r':
        if   size_f == 8 : r = reg8 [r]
        elif size_f == 16: r = reg16[r]
        elif size_f == 32: r = reg32[r]
        elif size_f == 64: r = reg64[r]
    elif r_f=='s': r = reg_s[r]
    # else nothing
    
    #mm
    if mod==0b11:
        if   size_f == 8 : mm = reg8 [m]
        elif size_f == 16: mm = reg16[m]
        elif size_f == 32: mm = reg32[m]
        elif size_f == 64: mm = reg64[m]
    elif mod==0b00:
        #rs # reg of seg
        #nt # nums tail
        if m==0b100: #<sib>
            if len(nums)<2: raise Exception('need <sib> byte')
            sib = nums[1]
            s = (sib & 0b11000000)>>6
            i = (sib & 0b111000)>>3
            b = sib & 0b111
            print('  s i b: {:02b} {:03b} {:03b}'.format(s,i,b))
            if b==0b101:
                return None
            rs = 'ss' if b==0b100 else 'ds' #???????????????
            nt=2
            mm = reg32[b]
            if i!=0b100:
                s = '*'+str(2**s)
                if s=='*1': s=''
                mm+='+'+reg32[i]+s
        elif m==0b101: 
            nt=1
            mm=''
            for k in range(4):
                if nt+k<len(nums):
                    mm = "{:02x}".format(nums[nt+k])+mm
                else:
                    mm = 'x'+str(nt+k-len(nums))+mm
            nt+=4
            rs = 'ds'
        else:
            nt=1
            rs = 'ds'
            mm=reg32[m]
        mm = rs+'['+mm+']'
        
    else:
        #rs # reg of seg
        #nt # nums tail
        if m==0b100: #<sib>
            if len(nums)<2: raise Exception('need <sib> byte')
            sib = nums[1]
            s = (sib & 0b11000000)>>6
            i = (sib & 0b111000)>>3
            b = sib & 0b111
            print('  s i b: {:02b} {:03b} {:03b}'.format(s,i,b))
            rs = 'ss' if b==0b100 or b==0b101 else 'ds' #???????????????
            nt=2
            mm = reg32[b]
            if i!=0b100:
                s = '*'+str(2**s)
                if s=='*1': s=''
                mm+='+'+reg32[i]+s
            else: return None
        else:
            nt=1
            rs = 'ss' if m==0b101 else 'ds'
            mm=reg32[m]
            
        tmp=''
        l = 1 if mod==0b01 else 4
        for k in range(l):
            if nt+k<len(nums):
                tmp = "{:02x}".format(nums[nt+k])+tmp
            else:
                tmp = 'x'+str(nt+k-len(nums))+tmp
        nt+=4
        mm = rs+'['+mm+('+'+tmp)+']'
        
    return (r,mm,nt-len(nums))
    

In [5]:
decode_arg('4c 85','n/rm32')

mod r m: 01 001 100
  s i b: 10 000 101


(1, 'ss[ebp+eax*4+x0]', 4)

In [90]:
decode_arg('04 24','n/rm32')

mod r m: 00 000 100
  s i b: 00 100 100


(0, 'ss[esp]', 0)

In [84]:
decode_arg('a1 01 00 00 00','n/rm32')

mod r m: 10 100 001


(4, 'ds[ecx+00000001]', 0)

In [85]:
decode_arg('45 03 00 00 00','n/rm32')

mod r m: 01 000 101


(0, 'ss[ebp+03]', 0)

In [9]:
hex(0x7710C1E1^0x76FDC550)

'0x1ed04b1'

# Introduction to the DWARF Debugging Format

It would be wonderful  if we could write
programs   that   were   guaranteed   to   work
correctly and never needed to be debugged.
Until   that   halcyon   day,   the   normal   programming cycle is going to involve writing
a program, compiling it, executing it, and
then  the  (somewhat)  dreaded  scourge  of
debugging it. And then repeat until the program works as expected. 

It is possible to debug programs by inserting code that prints values of selected
interesting variables. Indeed, in some situations, such as debugging kernel drivers, this
may   be  the   preferred   method.   There   are
low­level debuggers that allow you to step
through   the   executable   program,   instruction by instruction, displaying registers and
memory contents in binary. 

But it is much easier to use a source­level   debugger   which   allows   you   to   step
through   a   program's   source,   set   breakpoints, print variable values, and perhaps a
few other functions such as allowing you to
call a function in your program while in the
debugger. The problem is how to coordinate   two   completely   different   programs,
the compiler and the debugger, so that the
program can be debugged. 


## Translating from Source to Executable

The   process   of   compiling   a   program
from human­readable form into the binary form that a processor executes is quite
complex, but it essentially involves successively recasting the source into simpler and
simpler   forms,   discarding   information   at
each step until, eventually, the result is the
sequence   of   simple   operations,   registers,
memory   addresses,   and   binary   values
which the processor actually understands.
After all, the processor really doesn't care
whether you used object oriented programming, templates, or smart pointers; it only
understands a very simple set of operations
on a limited number of registers and memory locations containing binary values. 

As   a   compiler   reads   and   parses   the
source of a program, it collects a variety of
information about the program, such as the
line numbers where a variable or function
is declared or used. Semantic analysis extends this information to fill in details such
as the types of variables and arguments of
functions. Optimizations may move parts of
the   program   around,   combine   similar
pieces, expand inline functions, or remove
parts   which   are   unneeded.   Finally,   code
generation   takes   this   internal   representation of the program and generates the actual machine instructions. Often, there is another pass over the machine code to perform what are called "peephole" optimizations that may further rearrange or modify
the code, for example, to eliminate duplicate instructions. 

All­in­all, the compiler's task is to take
the well­crafted and understandable source
code and convert it into efficient but essentially unintelligible machine language. The
better the compiler achieves the goal of creating tight and fast code, the more likely it
is that the result will be difficult to understand. 

During   this   translation   process,   the
compiler   collects   information   about   the
program which will be useful later when
the  program  is debugged.  There   are two
challenges to doing this well. The first is
that in the later parts of this process, it may
be difficult for the compiler to relate the
changes it is making to the program to the
original source code that the programmer
wrote. For example, the peephole optimizer
may remove an instruction because it was
able to switch around the order of a test in
code that was generated by an inline function in the instantiation of a C++ template.
By the time it gets its metaphorical hands
on the program, the optimizer may have a
difficult time connecting its manipulations
of   low­level   code   to   the   original   source
which generated it.  

The second challenge is how to describe
the executable program and its relationship
to the original source with enough detail to
allow a debugger to provide the programmer useful information. At the same time,
the description has to be concise enough so
that it does not take up an extreme amount
of   space   or   require   significant   processor
time to interpret. This is where the DWARF
Debugging Format comes in: it is a compact
representation of the relationship between
the executable program and the source in a
way that is reasonably efficient for a debugger to process. 

## The Debugging Process
